# Le problème : la prévision de consommation électrique

Pour garantir l'équilibre offre-demande à chaque instant et gérer l'acheminement de l'électricité, RTE construit ses propres prévisions de la consommation nationale, régionale, et locale, à différentes échéances de temps (de l'infrajournalier au pluri-annuel).

Ici on se focalise sur un problème particulier : **la prévision de la consommation électrique nationale horaire à horizon J+1** (on suppose qu'on connaît toutes les données jusqu'au jour J inclus). 

## Les données : Eco2mix

La courbe de charge France est disponible sur eco2mix :
http://www.rte-france.com/fr/eco2mix/eco2mix
ou sur application mobile.

Vous pouvez naviguer sur le site pour vous familiariser avec les données sur lesquelles vous allez travailler.

## Objectif :

Au cours de ce TP, nous allons aborder les différentes étapes nécessaires à la construction d'un modèle de prévision de consommation :

1) Formalisation du problème: que souhaite-t-on prédire (quel est mon Y) ? Avec quelles variables explicatives (quel est mon X) ?

2) Collecte des données: où se trouvent les données ? Quel est le format ? Comment les récupérer ? (FACULTATIF - voir TP "TP1_Preparation_donnees")

3) Import des données et analyses descriptives : visualiser des séries temporelles, statistiques descriptives

4) Transformation des données (feature engineering) pour entrainer et tester un premier modèle

5) Création de prévision à dire d'expert pour servir de référence.

6) Découpage des données : apprentissage - test

7) Evaluer un modèle

8) Tester des algorithmes de référence : régression linéaire, forêts aléatoires, xgboost

9) Itérer à partir des modèles testés pour améliorer les prévisions

Nous verrons qu'une difficulté majeure réside dans la construction des "bonnes" variables explicatives ("garbage in, garbage out").

**Le notebook est parsemé de questions (<font color='green'>en vert</font>). Vous pouvez y répondre sur la feuille fournie.**

## Méthodes de prévision considérées

Les modèles actuels reposent sur des méthodes de régression linéaire et non-linéaire. Nous étudierons ici les limites de la régression linéaire.

Pour améliorer les prévisions, nous aurons recours aux méthodes dites de Machine Learning. Ces méthodes ne dépendent pas d'une formalisation a priori du lien entre les variables explicatives X et la variable à expliquer Y. 
Elles sont souvent moins interprétables mais peuvent être plus efficaces en prévision. Elles peuvent nécessiter plus de temps de calcul et plus de données pour cela.

Construire un bon modèle d'apprentissage nécessite en général de la connaissance experte dans le domaine d'intérêt pour créer des modèles pertinents et efficaces. 

## To be continued : deep learning

Le deuxième TP permettra d'investiguer les modèles "Deep" avec réseaux de neurones, en montrant le moindre besoin en feature engineering et leur plus grande capacité à absorber l'information grâce aux représentations hiérarchiques qu'ils créent.

# Environnement de travail 

Ceci est un notebook jupyter. Il permet d'exécuter du code python, d'afficher des résultats et d'écrire du texte pour décrire l'ensemble de l'étude.

<font color='red'>
    
**NB : L'aide de python est accessible en tapant help(nom_de_la_commande)**

</font>

## Chargement des packages

In [37]:
pip install pytz

Note: you may need to restart the kernel to use updated packages.


In [38]:
# Exécutez la cellule ci-dessous (par exemple avec shift-entrée)
# Si vous exécuter ce notebook depuis votre PC, il faudra peut-etre installer certaines librairies avec 
# 'pip install ma_librairie'
import os  # accès aux commandes système

import datetime  # structure de données pour gérer des objets calendaires
from datetime import timezone
import pandas as pd  # gérer des tables de données en python
import numpy as np  # librairie d'opérations mathématiques
from math import sqrt

import pytz

import sklearn  # librairie de machine learning
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, iplot_mpl
import matplotlib.pyplot as plt  # tracer des visualisations
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import zipfile  # compresser ou décompresser fichier

%matplotlib inline

%autosave 0

Autosave disabled


## Données disponibles

Choix du répertoire de travail "data_folder" dans lequel tous les fichiers csv seront entreposés. Ici le répertoire s'appelle *data*.

Ensuite on affiche les fichiers du répertoire pour vérification

In [39]:
data_folder = os.path.join(os.getcwd(), "data")

In [40]:
print("Mon repertoire est : {}".format(data_folder))
print("Fichiers contenus dans ce répertoire :")
for file in os.listdir(data_folder):
    print(" - " + file)

Mon repertoire est : D:\Users\montuelleluc\Documents\Formations-RTE\MOOC IA&DeepLearning\TP_Formation_Conso_DeepLearning\data
Fichiers contenus dans ce répertoire :
 - communes_coordonnees.csv
 - eCO2mix_RTE_tempo_2017-2018.xls
 - joursFeries.csv
 - meteoX_T0_T24.zip
 - StationsMeteoRTE.csv
 - Xinput.csv
 - Xinput.zip
 - Xtemperature.csv
 - Xtemperature.zip
 - Yconso.csv
 - YconsoT0.csv
 - Yconso_2014_2018.csv


# Récupération des données dans Python

Dans cette partie nous allons charger les fichiers csv nécessaires pour l'analyse, puis les convertir en data-frame python : 
- Yconso.csv
- Xinput.csv

Rappel : Les données brutes ont été pré-traitées à l'aide du notebook *TP1_Preparation_donnees.ipynb* pour obtenir ces deux fichiers.

## import de Yconso.csv

In [41]:
Yconso_csv = os.path.join(data_folder, "Yconso.csv")
Yconso = pd.read_csv(Yconso_csv)
display(Yconso.head(5)) # affichage des premières lignes
print(Yconso.dtypes) # affichage du type de données pour chaque colonne

,ds,y
0,2014-01-08 00:00:00+00:00,62008
1,2014-01-08 01:00:00+00:00,57298
2,2014-01-08 02:00:00+00:00,56216
3,2014-01-08 03:00:00+00:00,53719
4,2014-01-08 04:00:00+00:00,51798


ds    object
y      int64
dtype: object


La colonne "ds" contient la date, mais celle-ci n'est pas reconnue en tant que telle mais en tant que chaîne de caractères (https://pbpython.com/pandas_dtypes.html). On va la convertir en objet de type "datetime" plus approprié pour extraire des informations comme le jour de la semaine ou l'heure.  
Pour plus d'information, voir le TP1_Preparation_donnees.

In [42]:
Yconso['ds'] = pd.to_datetime(Yconso['ds'],utc=True)
print(Yconso.dtypes)
display(Yconso.head(5))

ds    datetime64[ns, UTC]
y                   int64
dtype: object


,ds,y
0,2014-01-08 00:00:00+00:00,62008
1,2014-01-08 01:00:00+00:00,57298
2,2014-01-08 02:00:00+00:00,56216
3,2014-01-08 03:00:00+00:00,53719
4,2014-01-08 04:00:00+00:00,51798


Visuellemement cela ne change rien.

On peut aussi afficher la dimension du DataFrame (toujours s'assurer que cela correspond aux valeurs attendues) : 

In [43]:
print(Yconso.shape)

(43368, 2)


## Import des variables d'entrée du modèle prédictif 

**Attention : Les données présentes dans Xinput sont encryptées dans un fichier zip.**  
Pour les lire vous avez besoin d'un mot de passe qui ne peut vous être donné que dans le cadre d'un travail au sein de RTE.

Sinon, la lecture se déroule comme pour le fichier Yconso.csv : transformation en datetime de la colonne *ds* et vérification des dimensions.

In [44]:
password = 'FIFA_2019'

In [45]:
Xinput_zip = os.path.join(data_folder, "Xinput.zip")
# Pour travailler avec les fichiers zip, on utilise la bibliothèque **zipfile**.
zipfile_xinput = zipfile.ZipFile(Xinput_zip)
zipfile_xinput.setpassword(bytes(password,'utf-8'))
Xinput = pd.read_csv(zipfile_xinput.open('Xinput.csv'),sep=",",engine='c',header=0)

Xinput['ds'] = pd.to_datetime(Xinput['ds'],utc=True)



Vous disposez de relevés de températures en stations (voir le fichier *data/StationsMeteoRTE.csv* pour plus d'informations) ainsi que d'une température France prévue pour l'instant considéré et celle réalisée la veille.

In [46]:
print("Dimensions de X")
print(Xinput.shape)
print("")
print("Colonnes de X")
print(Xinput.columns)
print("")
print("Aperçu de X")
display(Xinput.head(35))


Dimensions de X
(43368, 74)

Colonnes de X
Index(['ds', 'holiday', '002_0', '002_24', '005_0', '005_24', '015_0',
       '015_24', '027_0', '027_24', '070_0', '070_24', '110_0', '110_24',
       '120_0', '120_24', '130_0', '130_24', '145_0', '145_24', '149_0',
       '149_24', '156_0', '156_24', '168_0', '168_24', '180_0', '180_24',
       '190_0', '190_24', '222_0', '222_24', '240_0', '240_24', '255_0',
       '255_24', '260_0', '260_24', '280_0', '280_24', '299_0', '299_24',
       '434_0', '434_24', '460_0', '460_24', '481_0', '481_24', '497_0',
       '497_24', '510_0', '510_24', '579_0', '579_24', '588_0', '588_24',
       '621_0', '621_24', '630_0', '630_24', '643_0', '643_24', '645_0',
       '645_24', '650_0', '650_24', '675_0', '675_24', '690_0', '690_24',
       '747_0', '747_24', 'Th_real_24h_avant', 'Th_prev'],
      dtype='object')

Aperçu de X


,ds,holiday,002_0,002_24,005_0,005_24,015_0,015_24,027_0,027_24,...,650_0,650_24,675_0,675_24,690_0,690_24,747_0,747_24,Th_real_24h_avant,Th_prev
0,2014-01-08 00:00:00+00:00,NaN,10.30,9.69,10.00,9.80,9.69,9.30,10.19,10.50,...,10.8,12.90,5.60,10.60,8.50,9.50,10.7,11.20,9.846930,9.911160
1,2014-01-08 01:00:00+00:00,NaN,10.10,9.40,9.69,9.80,9.40,9.30,10.30,10.30,...,10.8,12.50,4.00,10.10,8.40,9.40,11.0,11.10,9.848500,9.790830
2,2014-01-08 02:00:00+00:00,NaN,10.00,9.19,9.90,9.60,9.19,9.40,10.30,10.00,...,10.6,12.30,4.40,9.60,7.90,9.30,11.6,11.00,9.681580,9.634990
3,2014-01-08 03:00:00+00:00,NaN,9.90,9.00,9.50,9.19,9.19,9.19,10.19,9.80,...,9.8,12.00,4.00,9.10,7.50,9.30,11.1,10.90,9.487130,9.445360
4,2014-01-08 04:00:00+00:00,NaN,9.90,8.80,9.00,8.90,8.90,9.00,10.19,9.60,...,11.1,11.80,3.00,8.80,7.30,9.00,10.8,10.80,9.490410,9.241585
5,2014-01-08 05:00:00+00:00,NaN,9.69,9.40,9.40,9.10,9.30,9.40,10.10,9.50,...,11.0,11.50,3.40,8.80,7.20,9.60,10.6,9.19,9.346880,9.045105
6,2014-01-08 06:00:00+00:00,NaN,9.60,9.19,9.60,8.90,9.19,9.10,10.30,9.50,...,11.9,11.60,4.50,8.90,7.20,9.40,10.5,9.00,9.284855,8.915265
7,2014-01-08 07:00:00+00:00,NaN,9.60,9.50,9.50,8.60,9.19,9.50,10.50,9.60,...,11.9,12.10,5.09,8.90,7.40,9.69,10.3,9.50,9.377060,8.964100
8,2014-01-08 08:00:00+00:00,NaN,9.90,9.40,9.19,8.60,9.30,9.50,10.70,9.50,...,12.1,12.50,5.80,9.40,8.00,9.60,10.3,9.90,9.543030,9.106500
9,2014-01-08 09:00:00+00:00,NaN,10.00,9.40,10.00,9.10,9.69,9.80,10.80,9.60,...,12.6,13.00,6.60,10.00,8.50,9.90,11.0,10.70,10.109460,9.630050


Dans cette étude de cas, par soucis de simplicité, nous allons travailler uniquement avec des
**températures France**. Utiliser les températures des différentes stations météo serait une piste
intéressante pour améliorer le modèle.

In [47]:
Xinput = Xinput[['ds', 'holiday', 'Th_real_24h_avant', 'Th_prev']]

<font color='green'>
    
* Quelles sont les variables disponibles (dans Xinput et Yconso)?
    
* Quelles sont les dimensions (nombre d’observations et de variables) de Xinput et Yconso après lecture des fichiers csv? Est-ce cohérent?

* Les données présentes dans Xinput vous semblent-elles pertinentes pour prédire la consommation nationale présente dans Yconso ?

* Que pensez-vous de cette notion de "Température France"?
</font>

# Visualisation des données 

La DataScience et le Machine Learning supposent de bien appréhender les données sur lesquelles nos modèles vont être entrainés. Pour cela, il est utile de faire des statistiques descriptives et des visualisations de nos différentes variables.

Traitant d'un problème de prévision, on visualisera en particulier des séries temporelles.

Dans les slides d'introduction, vous allez voir des :
- échantillons de données
- profils de courbe de consommation journaliers et saisonniers
- visualisations de corrélation entre conso J et conso retardée


# Outils de construction de modèle
<img src="pictures/etabli.jpg"  width=500 height=60>

## Construction des jeux d'entrainement et de test

Pour éviter de construire un modèle qui apprend "par coeur" sur ses données, et qui disposerait alors d'une capacité de généralisation faible, il est d'usage courant de disposer de plusieurs jeux de données (de caractéristiques similaires). Le minimum est de construire 

* un jeu d'entraînement, sur lequel on estime le modèle, 

* un jeu de test, jamais vu durant l'entraînement, sur lequel on va évaluer le modèle. 

Rapidement dit : un bon modèle est un modèle dont la capacité prédictive ne se dégrade pas trop sur le jeu test.

Pour cela, on crée la fonction *prepareDataSetEntrainementTest* qui va permettre de couper Y et Xinput en deux parties.

In [ ]:
def prepareDataSetEntrainementTest(Xinput, Yconso, dateDebut, dateRupture, nbJourlagRegresseur=0):
    
    dateStart = Xinput.iloc[0]['ds']
    
    DateStartWithLag = dateStart + pd.Timedelta(str(nbJourlagRegresseur)+' days')  #si un a un regresseur avec du lag, il faut prendre en compte ce lag et commencer l'entrainement a la date de debut des donnees+ce lag
    XinputTest = Xinput[(Xinput.ds >= dateRupture)]    

    XinputTrain=Xinput[(Xinput.ds < dateRupture) & (Xinput.ds > DateStartWithLag) & (Xinput.ds > dateDebut)]
    YconsoTrain=Yconso[(Yconso.ds < dateRupture) & (Yconso.ds > DateStartWithLag) & (Yconso.ds > dateDebut)]
    YconsoTest=Yconso[(Xinput.ds >= dateRupture)]
    
    return XinputTrain, XinputTest, YconsoTrain, YconsoTest

## Fonctions utilitaires

Créons la fonction modelError qui va calculer pour un échantillon (Y, Y_hat) différents scores :
- erreur relative moyenne (MAPE en %)
- erreur relative max (en %)
- RMSE (en MW)

Cette fonction est ensuite utilisée par les fonctions *evaluation* et *evaluation_par* qui nous permettront d'évaluer nos modèles.

<img src="pictures/evaluation.jpg"  width=250 height=30>

In [ ]:
def modelError(Y, Yhat):

    Y = Y.reset_index(drop=True)
    
    relativeErrorsTest = np.abs((Y['y'] - Yhat) /Y['y']) 
    errorMean = np.mean(relativeErrorsTest)
    errorMax = np.max(relativeErrorsTest)
    rmse = np.sqrt(mean_squared_error(Y['y'], Yhat))
   
    return relativeErrorsTest, errorMean, errorMax, rmse

In [ ]:
def evaluation(YTrain, YTest, YTrainHat, YTestHat):
    # Ytrain et Ytest ont deux colonnes : ds et y
    # YtrainHat et YTestHat sont des vecteurs
    ErreursTest, ErreurMoyenneTest, ErreurMaxTest, RMSETest = modelError(YTest, YTestHat)
    print("l'erreur relative moyenne de test est de:" + str(round(ErreurMoyenneTest*100,1))+"%")
    print("l'erreur relative max de test est de:" + str(round(ErreurMaxTest*100,1)) +"%")
    print('le rmse de test est de:' + str(round(RMSETest,0)))
    print()
    ErreursTest, ErreurMoyenneTest, ErreurMaxTest, RMSETest = modelError(YTrain, YTrainHat)
    print("l'erreur relative moyenne de train est de:" + str(round(ErreurMoyenneTest*100,1))+"%")
    print("l'erreur relative max de train est de:" + str(round(ErreurMaxTest*100,1)) +"%")
    print('le rmse de test est de:' + str(round(RMSETest,0))) 

In [ ]:
def evaluation_par(X, Y, Yhat,avecJF=True):
    Ytmp = Y
    Ytmp['weekday'] = Ytmp.ds.dt.weekday
    Ytmp['hour'] = Ytmp.ds.dt.hour
    if(avecJF):
        Ytmp['JoursFeries'] = X['JoursFeries']
    Ytmp['APE'] = np.abs(Ytmp['y']-Yhat)/Ytmp['y']
    dataWD = Ytmp[['weekday','APE']]
    groupedWD = dataWD.groupby(['weekday'], as_index=True)
    statsWD = groupedWD.aggregate([np.mean])
    dataHour = Ytmp[['hour','APE']]
    groupedHour = dataHour.groupby(['hour'], as_index=True)
    statsHour = groupedHour.aggregate([np.mean])
    
    if(avecJF):
        dataJF = Ytmp[['JoursFeries','APE']]
        groupedJF = dataJF.groupby(['JoursFeries'], as_index=True)
        statsJF = groupedJF.aggregate([np.mean])
    else:
        statsJF = None
    
    return statsWD, statsHour, statsJF


# Feature engineering : préparation de Xinput

L'objectif de cette partie est d'enrichir Xinput à partir des données initiales. Il s'agit notamment d'exploiter les différentes informations calendaires disponibles.

Dans un premier temps, on supprime la consommation retardée d'une heure, non disponible pour notre exercice de prévision J+1.
Et on vérifie le travail.


In [ ]:
Xinput = Xinput.drop(['lag1H'],axis=1)  

In [ ]:
print(Xinput.shape)
print(Xinput.columns)

Ensuite, on encode les données calendaires en one-hot encoding pour le modèle. Autrement dit, on construit pour chaque modalité, une variable binaire associée.
Cet encodage est nécessaire pour que le modèle mathématique puisse appréhender la notion de date.

In [ ]:
encodedWeekDay = pd.get_dummies(Xinput['weekday'],prefix="weekday")
encodedMonth = pd.get_dummies(Xinput['month'],prefix="month")
encodedHour = pd.get_dummies(Xinput['hour'],prefix="hour")

In [ ]:
encodedWeekDay.head(3)

In [ ]:
encodedMonth.head(3)

In [ ]:
encodedHour.head(3)

In [ ]:
Xinput = pd.concat([Xinput, encodedMonth, encodedWeekDay, encodedHour], axis=1)
Xinput = Xinput.drop(['month','weekday','hour'],axis=1)

In [ ]:
print(Xinput.shape)
print(Xinput.columns)

Pour les données météo, on récupère aussi la prévision effectuée pour la veille. 

In [ ]:
colsToKeepWeather = [s for s in Xinput.columns.get_values() if 'Th_prev' in s]
lag_colsToKeepWeather = [ s + "_J_1" for s in colsToKeepWeather ]
Xinput[lag_colsToKeepWeather] = Xinput[colsToKeepWeather].shift(24)


On récupère aussi le jour de l'année grâce à la fonction *dayofyear*, ainsi que les jours fériés. On crée une variable binaire associée à chaque jour férié.

In [ ]:
#Récupération des jours fériés dans Xinput
time = pd.to_datetime(Xinput['ds'], yearfirst=True,utc=True)
Xinput['posan']= time.dt.dayofyear
encodedHolidays = pd.get_dummies(Xinput[['holiday']], prefix = "JF")
encodedHolidays['JoursFeries'] = encodedHolidays.sum(axis = 1)
Xinput = pd.concat([Xinput, encodedHolidays], axis = 1)
Xinput = Xinput.drop(['holiday'], axis = 1)

On ajoute des températures seuillées, à 15°C pour l'effet chauffage, et à 18°C pour l'effet climatisation.

In [ ]:
threshold_temperature_heat = 15
threshold_temperature_cool = 18

Xinput['temp_prev_with_threshold_heat'] = np.maximum(0, threshold_temperature_heat - Xinput['Th_prev'].values)
Xinput['temp_prev_with_threshold_cool'] = np.maximum(0, Xinput['Th_prev'].values - threshold_temperature_cool)

In [ ]:
#affichage de toutes les variables de base
list(Xinput) #list plutôt que print pour avoir la liste complète

Enfin, nous construisons les listes pour appeler plus rapidement les colonnes d'un même type.

In [ ]:
colsToKeepWeather = [s for s in Xinput.columns.get_values() if 'Th_prev' in s]
colsToKeepMonth = [v for v in Xinput.columns.get_values() if 'month' in v]
colsToKeepWeekday = [v for v in Xinput.columns.get_values() if 'weekday' in v]
colsToKeepHour = [v for v in Xinput.columns.get_values() if 'hour' in v]
colsToKeepHolidays = [v for v in Xinput.columns.get_values() if 'JF_' in v]

In [ ]:
# on souhaite un jeu de test qui commence à partir du 1er mai 2017
dateDebut = pytz.utc.localize( datetime.datetime(year=2014, month=1, day=15))#pour éviter les NaN dans le jeu de données
dateRupture = pytz.utc.localize(datetime.datetime(year=2017, month=12, day=1))#début du challenge prevision de conso
nbJourlagRegresseur = 0

In [ ]:
Yconso.tail()

In [ ]:
XinputTrain, XinputTest, YconsoTrain, YconsoTest = prepareDataSetEntrainementTest(Xinput, Yconso, 
                                                                                  dateDebut, dateRupture, 
                                                                                  nbJourlagRegresseur)

In [ ]:
print('la taille de l échantillon XinputTrain est:' + str(XinputTrain.shape))
print('la taille de l échantillon XinputTest est:' + str(XinputTest.shape))
print('la taille de l échantillon YconsoTrain est:' + str(YconsoTrain.shape))
print('la taille de l échantillon YconsoTest est:' + str(YconsoTest.shape))
print("la proportion de data d'entrainement est de:" + str(YconsoTrain.shape[0] / (YconsoTrain.shape[0] + YconsoTest.shape[0])))

# Construction d'un modèle prédictif naïf

<img src="pictures/hommeNaif.png" width=500 height=60>

## Première idée, un modèle naïf : on plaque bêtement la valeur de consommation nationale de la veille

On commence par juste notre point horaire

In [ ]:
y_pred_modele_naif_1 = float(Xinput.loc[Xinput['ds'] == datetime_a_predire]['lag1D'])
pred_error = abs(y_true_point_horaire_cible - y_pred_modele_naif_1)

print("Modele 1 -- pred: {}, realisee: {}, erreur: {}%".format(y_pred_modele_naif_1, y_true_point_horaire_cible, pred_error/y_true_point_horaire_cible * 100))

Voyons maintenant ce que ça donne non plus sur un unique point horaire mais sur l'ensemble des points horaires :

In [ ]:
y_pred_modele_naif_1 = Xinput["lag1D"]

pred_error = (np.abs(Yconso["y"] - y_pred_modele_naif_1.loc[24:]) / Yconso["y"] * 100)

print(np.mean(pred_error))

In [ ]:
pred_train_naif1= XinputTrain["lag1D"]
pred_test_naif1= XinputTest["lag1D"]
evaluation(YconsoTrain, YconsoTest,  pred_train_naif1.values,pred_test_naif1.values)


Bon c'est pas fou...

## Deuxième idée : modèle naïf avec de l'expertise métier 

Chez RTE, on considère qu'une augmentation moyenne de 1°C conduit à une augmentation de 2400MW de la consommation nationale pour des températures inférieures à 15°C. On propose donc comme consommation prévue la consommation de la veille, corrigée par 2400 fois l'écart à la température de la veille, si l'on n'excède pas les 15°C.


<img src="pictures/ExpertJamy.jpg" width=500 height=60>

In [ ]:
delta_MW_par_degre = 2400  
            
threshold_temperature = 15

On commence par juste notre point horaire préféré

In [ ]:
temperature_real_veille = float(Xinput.loc[Xinput['ds'] == datetime_a_predire]['Th_real_24h_avant'])
temperature_prevu_cible = float(Xinput.loc[Xinput['ds'] == datetime_a_predire]['Th_prev'])
delta_temp = min(threshold_temperature, temperature_prevu_cible) - min(threshold_temperature, temperature_real_veille)
delta_MW_because_temp = delta_temp * delta_MW_par_degre

y_pred_modele_naif_2 = float(Xinput.loc[Xinput['ds'] == datetime_a_predire]['lag1D']) - delta_MW_because_temp
pred_error = abs(y_true_point_horaire_cible - y_pred_modele_naif_2)

print("Modele 2 -- pred: {}, realisee: {}, erreur: {}%".format(y_pred_modele_naif_2, y_true_point_horaire_cible, pred_error/y_true_point_horaire_cible * 100))

Et maintenant sur l'ensemble des points horaires :

In [ ]:
y_pred = Xinput["lag1D"]

temp_prev_with_threshold = np.minimum([threshold_temperature], Xinput['Th_prev'].values)
temp_actual_with_threshold = np.minimum([threshold_temperature], Xinput['Th_real_24h_avant'].values)

delta_temp = temp_prev_with_threshold - temp_actual_with_threshold
delta_MW_because_temp = delta_temp * delta_MW_par_degre

y_pred_modele_naif_2 = Xinput["lag1D"] - delta_MW_because_temp
pred_error = (np.abs(Yconso["y"] - y_pred_modele_naif_2) / Yconso["y"] * 100)

print(np.mean(pred_error))

Bon... Bien essayé avec ces modèles naïfs, mais maintenant on va être plus sérieux !

## Test des fonctions sur le modèle expert

Recalculons les prévisions à l'aide des deux modèles experts et évaluons ces deux approches :

### Modèle naïf 1

### Modèle naïf 2

In [ ]:
# prévision train
temp_prev_with_threshold = np.minimum([threshold_temperature], XinputTrain['Th_prev'].values)
temp_actual_with_threshold = np.minimum([threshold_temperature], XinputTrain['Th_real_24h_avant'].values)

delta_temp = temp_prev_with_threshold - temp_actual_with_threshold
delta_MW_because_temp = delta_temp * delta_MW_par_degre

pred_train_naif2 = XinputTrain["lag1D"] - delta_MW_because_temp


# prévision test
temp_prev_with_threshold = np.minimum([threshold_temperature], XinputTest['Th_prev'].values)
temp_actual_with_threshold = np.minimum([threshold_temperature], XinputTest['Th_real_24h_avant'].values)

delta_temp = temp_prev_with_threshold - temp_actual_with_threshold
delta_MW_because_temp = delta_temp * delta_MW_par_degre
pred_test_naif2 = XinputTest["lag1D"] - delta_MW_because_temp

# scores
evaluation(YconsoTrain, YconsoTest,  pred_train_naif2.values,pred_test_naif2.values)



# Régression linéaire simple

Le modèle naïf avec expertise métier a été inspiré de la forme de la courbe d'évolution de la consommation en fonction de la température en France. 
Pour rappel:

In [ ]:
plt.scatter(Xinput['Th_prev'], Yconso['y'], alpha=0.2)
plt.show()

La consommation pourrait être modélisée par une fonction linéaire par morceaux de la température, avec une pente plus importante pour les températures froides que pour les températures élevées. Au lieu de fixer les gradients à 2400MW/°C et 0, ceux-ci pourraient être calibrés à partir des données.


## Entrainer un modèle
Notre modèle a des paramètres qu'il va falloir maintenant apprendre au vu de notre jeu d'entrainement. Il faut donc caler notre modèle sur ce jeu d'entrainement.

In [ ]:
colsLR_simple = np.concatenate(([s for s in XinputTrain.columns.get_values() if 'temp_prev_with_' in s], colsToKeepHour, colsToKeepWeekday, colsToKeepMonth))

mTrain = linear_model.LinearRegression(fit_intercept = False)
mTrain.fit(XinputTrain[colsLR_simple], YconsoTrain[['y']])


## Interpréter le modèle 

In [ ]:
coef_joli = pd.DataFrame(np.concatenate(( np.array([colsLR_simple]).T,mTrain.coef_.T),axis=1),columns = ['variable','coefficient'])
print(coef_joli)


<font color='green'>

* Commentez les coefficients de régression obtenus. 
* Comparez notamment les gradients obtenus avec le modèle naïf.

</font>

## Faire des prédictions
Une fois qu'un modèle de prévision est entrainé, il ne s'avère utile que s'il est performant sur de nouvelles situations. Faisons une prévision sur notre jeu de test. Traçons les courbes obtenues et calculons les scores.

In [ ]:
forecastTest = np.concatenate(mTrain.predict(XinputTest[colsLR_simple]))
forecastTrain = np.concatenate(mTrain.predict(XinputTrain[colsLR_simple]))

In [ ]:
plt.scatter(forecastTest, YconsoTest[['y']])
plt.show()

plt.plot(YconsoTest['ds'], YconsoTest['y'], 'b', YconsoTest['ds'], forecastTest, 'r')
plt.show()

In [ ]:
evaluation(YconsoTrain, YconsoTest, forecastTrain,  forecastTest)
evalWD,evalHour,evalJF = evaluation_par(XinputTest,YconsoTest,forecastTest,avecJF=True)
print(str(round(evalWD*100,1)))
print(str(round(evalHour*100,1)))
print(str(round(evalJF*100,1)))

### Comment se distribue l'erreur ?

In [ ]:
erreur_relative_test, erreur_moyenne_test, erreur_max_test, rmse = modelError(YconsoTest, forecastTest)

In [ ]:
num_bins = 100
plt.hist(erreur_relative_test, num_bins)
plt.show()

### A quel moment se trompe-t-on le plus ?

In [ ]:
plt.plot(YconsoTest['ds'], erreur_relative_test, 'r')
plt.title("erreur relative sur la periode de test")
plt.show()

In [ ]:
erreur_relative_test.head()

In [ ]:
threshold = 0.18

mask = (erreur_relative_test >= threshold)
erreurs_df = pd.DataFrame(np.concatenate((YconsoTest[['ds','y']],np.array([forecastTest]).T),axis=1),columns=["date","y","prev"])
print(erreurs_df[mask])


Au vu des résultats précédents :
<font color= 'green'>

- que pensez-vous du modèle?
- comment se distribue l'erreur?
- quand se trompe-t-on le plus?
- quelles variables explicatives ajouter?

</font>

# Autres modèles : RandomForest et XGBoost

## Modèle RandomForest

<img src="pictures/randomForestExplain.png" width=500 height=30>

### Choix des données d'entrée

In [ ]:
colsRF = np.concatenate((['lag1D','lag1W'],
                         colsToKeepWeather,colsToKeepMonth,colsToKeepWeekday,colsToKeepHour,colsToKeepHolidays))
list(colsRF)

### Entrainement du modèle

In [ ]:
# La cellule peut prendre un peu de temps à exécuter
print(Xinput.head(20))
rfTrain = RandomForestRegressor(n_estimators=30, max_features=colsRF.size, n_jobs=3, oob_score = True, bootstrap = True)
rfTrain.fit(XinputTrain[colsRF], YconsoTrain['y'])

<font color='green'>

* Grâce à l'aide de la fonction, déterminer les paramètres de cette méthode

</font>

### Prediction

In [ ]:
forecastTest = rfTrain.predict(XinputTest[colsRF])
forecastTrain = rfTrain.predict(XinputTrain[colsRF])

### Evaluation

In [ ]:
evaluation(YconsoTrain, YconsoTest, forecastTrain, forecastTest)

# on visualise nos previsions par rapport a la realité
plt.plot(YconsoTest['ds'], YconsoTest['y'], 'b', YconsoTest['ds'], forecastTest, 'r')
plt.show()

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rfTrain.score(XinputTrain[colsRF], YconsoTrain['y']), 
                                                                                             rfTrain.oob_score_,
rfTrain.score(XinputTest[colsRF], YconsoTest['y'])))

In [ ]:
evalWD,evalHour,evalJF = evaluation_par(XinputTest,YconsoTest,forecastTest)
print(str(round(evalWD*100,1)))
print(str(round(evalHour*100,1)))
print(str(round(evalJF*100,1)))

## Modèle xgboost

<img src="pictures/XGboost.png" width=500 height=30>

In [ ]:
xgbTrain = xgb.XGBRegressor(objective='reg:squarederror')
xgbTrain.fit(XinputTrain[colsRF], YconsoTrain['y'].values)
forecastTestXGB = xgbTrain.predict(XinputTest[colsRF])
forecastTrainXGB = xgbTrain.predict(XinputTrain[colsRF])

<font color='green'>

* Grâce à l'aide de la fonction, déterminer les paramètres de cette méthode

</font>

In [ ]:
evaluation(YconsoTrain, YconsoTest, forecastTrainXGB, forecastTestXGB)

In [ ]:
evalWD,evalHour,evalJF = evaluation_par(XinputTest, YconsoTest, forecastTestXGB)
print(str(round(evalWD * 100,1)))
print(str(round(evalHour * 100,1)))
print(str(round(evalJF * 100,1)))

# Bonus: à vous de jouer

Bravo ! Vous avez déjà créé un premier modèle performant pour faire des prévisions sur une fenêtre glissante à horizon 24h !

Maintenant à vous de mettre votre expertise pour créer de nouveaux modèles.

Vous pouvez continuer à explorer le problème selon plusieurs axes:
- choix de la méthode de modélisation
- apprendre votre modèle sur une période différente
- créer de nouvelles variables explicatives ? Quid de la météo et de la température? Des jours fériés ? Du feature engineering plus complexe...
- détecter des outliers dans les données

Mettez-vous en 3 groupes, explorez pendant 30 minutes, et restituez.